In [1]:
%pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [2]:
import seamless
from seamless.highlevel import Context, Transformer, Cell
import numpy as np

seamless.delegate(False)
ctx = Context()

In [3]:
ctx.pdb1 = open("1AKE-flanks.pdb").read()
ctx.pdb2 = open("1AKE-B-hit.pdb").read()
ctx.load_pdb1 = Transformer()
ctx.load_pdb1.pdb = ctx.pdb1
ctx.load_pdb_code = ctx.load_pdb1.code.pull()
ctx.load_pdb_code.mount("load_pdb.py")
ctx.flanks = ctx.load_pdb1

ctx.load_pdb2 = Transformer()
ctx.load_pdb2.pdb = ctx.pdb2
ctx.load_pdb2.code = ctx.load_pdb_code
ctx.dbca = ctx.load_pdb2

environment_yml = """
channels:
- bioconda
- conda-forge
dependencies:
- biopython
"""
ctx.load_pdb1.environment.set_conda(environment_yml, "yaml")
ctx.load_pdb2.environment.set_conda(environment_yml, "yaml")
await ctx.computation()
print(ctx.load_pdb1.logs)
print(ctx.load_pdb2.logs)

*************************************************
* Result
*************************************************
<binary buffer of length 320, checksum fe4621ada9594853b8c9c13e6fd0b63681df0e8b209079f57b2b74a220f58a94>
*************************************************
Execution time: 0.0 seconds
*************************************************
*************************************************
* Result
*************************************************
<binary buffer of length 488, checksum 32a00701fd118918d48c69e37599a11a7ac8cf4e813e4dab10276367e844b3ce>
*************************************************
Execution time: 0.0 seconds
*************************************************


In [4]:
ctx.get_flank1 = lambda flanks: flanks[:4]
ctx.get_flank1.flanks = ctx.flanks
ctx.flank1 = ctx.get_flank1

ctx.get_flank2 = lambda flanks: flanks[-4:]
ctx.get_flank2.flanks = ctx.flanks
ctx.flank2 = ctx.get_flank2

await ctx.computation()
print(ctx.flank1.value)
print(ctx.flank2.value)

<Silk: [[26.3220005  54.8409996  13.68799973]
 [28.35499954 57.76399994 15.05799961]
 [26.33300018 57.77600098 18.23900032]
 [25.08600044 54.26399994 18.40200043]] >
<Silk: [[10.22099972 53.05599976 17.74900055]
 [10.58399963 55.20399857 14.63700008]
 [12.63899994 58.48400116 14.32499981]
 [10.53299999 61.81999969 14.44999981]] >


```
int BCLoopSearch (const Coord *atoms1, int nr_atoms1, const Coord *atoms2, int nr_atoms2,  //flank1 and flank2
                  int looplength, //size of the gap/loop we are searching
                  int minloopmatch, int maxloopgap, //for partial matches: minimum total length, maximum gap
                  int mirror, //looking for mirrors?
                  float minBC, float maxR, //minimum BC score, maximum rigidity
                  const Coord *dbca, //CA database
                  int seg_index[][3], //(dbca offset, segment resnr, segment length)
                  int pdb_index[][2], int nr_pdbindex, //(seg_index offset, number of segments), total number of PDBs
                  int hits[][3], //pdbindex line, seg_index line, segment offset
                  float hitstats[][2] //score, rigidity
                 )
{
```

In [5]:
ctx.bcloopsearch = Transformer()
ctx.bcloopsearch.language = "c"
ctx.bcloopsearch.main_module.compiler_verbose = False
ctx.bcloopsearch.code.mount("bcloopsearch.c", authority="file")
ctx.bcloopsearch.main_module.lib.language = "c"
ctx.bclib_code = ctx.bcloopsearch.main_module.lib.code.pull()
ctx.bclib_code.mount("BCLoopSearch-lib.c", authority="file")

ctx.bc_hits = ctx.bcloopsearch
await ctx.translation()

('bcloopsearch', '_main_module', 'lib', 'code'): cannot detect language, default to c.


In [6]:
def set_example(bc):
    bc.atoms1 = np.zeros((4, 3))
    bc.atoms2 = np.zeros((4, 3))
    bc.looplength = 5
    bc.minloopmatch = 5
    bc.maxloopgap = 0
    bc.mirror = False
    bc.minBC = 0.9
    bc.maxR = 9999
    bc.dbca = np.zeros((10, 3))
    bc.seg_index = np.zeros((10,3), dtype=np.uint32)
    bc.pdb_index = np.zeros((10,2), dtype=np.uint32)

set_example(ctx.bcloopsearch.example)

schema = ctx.bcloopsearch.schema
schema.properties.atoms1["form"].contiguous = True
schema.properties.atoms1["form"].shape = (-1, 3)
schema.properties.atoms2["form"].contiguous = True
schema.properties.atoms2["form"].shape = (-1, 3)
schema.properties.dbca["form"].shape = (-1, 3)
schema.properties.dbca["form"].contiguous = True
schema.properties.seg_index["form"].shape = (-1, 3)
schema.properties.seg_index["form"].contiguous = True
schema.properties.pdb_index["form"].shape = (-1, 2)
schema.properties.pdb_index["form"].contiguous = True

MAXHITS = 100000
bcrx = ctx.bcloopsearch.result.example
bcrx.nhits = 0
bcrx.hits = np.zeros((MAXHITS,3), dtype=np.uint32)
bcrx.hitstats = np.zeros((MAXHITS,2), dtype=np.float32)
await ctx.computation()
rschema = ctx.bcloopsearch.result.schema
rschema.properties.hits["form"].shape = (MAXHITS, 3)
rschema.properties.hitstats["form"].shape = (MAXHITS, 2)
await ctx.computation()

In [7]:
set_example(ctx.bcloopsearch)
ctx.bcloopsearch.atoms1 = ctx.flank1
ctx.bcloopsearch.atoms2 = ctx.flank2
ctx.bcloopsearch.dbca = ctx.dbca
ctx.bcloopsearch.seg_index = np.array([[0,1,len(ctx.dbca.value)]],dtype=np.uint32)
ctx.bcloopsearch.pdb_index = np.array([[0,1]],dtype=np.uint32)
ctx.bcloopsearch.looplength = 7
ctx.bcloopsearch.minBC = 0
await ctx.computation()

In [8]:
print(ctx.bcloopsearch.schema)
print()
print(ctx.bcloopsearch.result.schema)
print()
print(ctx.bcloopsearch.status)
print(ctx.bcloopsearch.exception)

{'properties': {'atoms1': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 8, 'type': 'number'}}, 'storage': 'binary', 'type': 'array'}, 'atoms2': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 8, 'type': 'number'}}, 'storage': 'binary', 'type': 'array'}, 'dbca': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 8, 'type': 'number'}}, 'storage': 'binary', 'type': 'array'}, 'looplength': {'type': 'integer'}, 'maxR': {'type': 'integer'}, 'maxloopgap': {'type': 'integer'}, 'minBC': {'type': 'number'}, 'minloopmatch': {'type': 'integer'}, 'mirror': {'type': 'boolean'}, 'pdb_index': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 2]}, 'items': {'form': {'bytesize': 4, 'type': 'integer', 'unsigned': True}}, 'storage': 'binary', 'type': 'array'}, 'seg_index': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 4, 'type': '

In [9]:
ctx.header = ctx.bcloopsearch.header
ctx.header.mimetype = "h"
ctx.header.output()

Output()

In [10]:
ctx.bcloopsearch_schema = Cell()
ctx.bcloopsearch_schema.celltype = "plain"
ctx.link(ctx.bcloopsearch_schema, ctx.bcloopsearch.inp.schema)
ctx.bcloopsearch_schema.mount("bcloopsearch-schema.json")
await ctx.computation()

In [11]:
ctx.bc_hits.value

<Silk: {'hits': array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint32), 'hitstats': array([[0.9981744 , 0.41393602],
       [0.        , 0.        ],
       [0.        , 0.        ],
       ...,
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]], dtype=float32), 'nhits': 1} >

In [12]:
nhits = ctx.bc_hits.value.nhits
print(nhits)

<Silk: 1 >


In [13]:
ctx.bc_hits.value.unsilk["hitstats"][:nhits]

array([[0.9981744 , 0.41393602]], dtype=float32)

In [14]:
ctx.pdb2.set(open("1AKE-B.pdb").read())
await ctx.computation()
ctx.bcloopsearch.seg_index= np.array([[0,1,len(ctx.dbca.value)]],dtype=np.uint32)
await ctx.computation()
ctx.bcloopsearch.minBC = 0.7
await ctx.computation()

In [15]:
nhits = ctx.bc_hits.value.nhits
print(nhits)

<Silk: 4 >


In [16]:
ctx.bc_hits.value.unsilk["hitstats"][:nhits]

array([[ 0.8497782 ,  6.7795157 ],
       [ 0.7490894 ,  9.142373  ],
       [ 0.9981744 ,  0.41393602],
       [ 0.70380884, 10.4923525 ]], dtype=float32)

In [17]:
dbca = np.load("db/scop-g.npy")[:, 1].astype(np.float64)
ctx.load_db_index = lambda pdbindex, segindex: None
ctx.load_db_index.pdbindex = open("db/scop-g.pdbindex").read()
ctx.load_db_index.segindex = open("db/scop-g.segindex").read()
ctx.load_db_index.code.mount("load_db_index.py", authority="file")
ctx.db_index = ctx.load_db_index
del ctx.dbca
ctx.dbca = dbca
ctx.bcloopsearch.dbca = ctx.dbca
ctx.bcloopsearch.seg_index = ctx.db_index.seg
ctx.bcloopsearch.pdb_index = ctx.db_index.pdb
ctx.bcloopsearch.minBC = 0.99
await ctx.computation()

In [18]:
nhits = ctx.bc_hits.value.nhits
print(nhits)

<Silk: 17 >


In [19]:
pdbs = ctx.bc_hits.value.unsilk["hits"][:nhits,0]
print(np.take(ctx.db_index.value.unsilk["pdb_names"], pdbs))
print(ctx.bc_hits.value.unsilk["hits"][:nhits,1])
print(ctx.bc_hits.value.unsilk["hits"][:nhits,2])
print(ctx.bc_hits.value.unsilk["hitstats"][:nhits])

['d1zipa2' 'd2akya2' 'd1dvra2' 'd1e4ya2' 'd1e4yb2' 'd1e4va2' 'd1e4vb2'
 'd3hpqa2' 'd3hpqb2' 'd1akea2' 'd1akeb2' 'd1anka2' 'd4akea2' 'd4akeb2'
 'd2ecka2' 'd2eckb2' 'd2ar7a2']
[25029 25033 25035 25041 25042 25043 25044 25045 25046 25047 25048 25049
 25051 25052 25053 25054 25057]
[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4]
[[0.99150455 0.9326091 ]
 [0.9926247  0.07179447]
 [0.9944583  0.20403697]
 [0.9960647  0.5190825 ]
 [0.9971652  0.68993837]
 [0.99814665 0.08378951]
 [0.99811184 0.2650473 ]
 [0.9989707  0.11703701]
 [0.99860483 0.18700984]
 [1.         0.        ]
 [0.9981703  0.41412714]
 [0.9970384  0.18814881]
 [0.99861157 0.297172  ]
 [0.9949829  0.402861  ]
 [0.99926007 0.08981241]
 [0.9987744  0.02204398]
 [0.99002814 0.2134962 ]]
